# Importing libraries

In [1]:
import pickle
import numpy as np
import pandas as pd 
pd.set_option("display.max_columns", None)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_squared_error, accuracy_score

# Loading and Preparing the Dataframe

In [2]:
data = pd.read_csv("../Data/Clean/data_cleaned.csv")
data

,season,season_start_year,league,division,league_finish,position,name,nationality,age,minutes_(total),starting_appearances_(league),substitute_appearances_(league),starting_appearances_(total),substitute_appearances_(total),total_appearances,goals_(total),goals_(league),g/minutes_(total),g/appearances_(total),european_competition,tier,performance,round_of_tournament,homegrown_players,transfers_in,transfers_out,net_transfers,games_played_(total),goals_for_(total),goals_against_(total),goal_difference_(total),goals_per_game_(total),games_played_(league),goals_for_(league),goals_against_(league),goal_difference_(league),goals_per_game_(league),average_age,average_appearances,average_minutes
0,1995-1996,1995,Segunda División,2,15,GK,Luis Pascual,Spain,27,2880,31,0,32,0,32,0,0,0.000000,0.000000,DNQ,DNQ,DNQ,NaN,2,6,4,2.0,40,36,43,-7,0.900000,38,32,39,-7,0.842105,25.60,17.79,1240.42
1,1995-1996,1995,Segunda División,2,15,GK,Alejandro Revuelta,Spain,24,0,0,0,0,0,0,0,0,0.000000,0.000000,DNQ,DNQ,DNQ,NaN,2,6,4,2.0,40,36,43,-7,0.900000,38,32,39,-7,0.842105,25.60,17.79,1240.42
2,1995-1996,1995,Segunda División,2,15,GK,Juan Díez de los Ríos,Spain,19,0,0,0,0,0,0,0,0,0.000000,0.000000,DNQ,DNQ,DNQ,NaN,2,6,4,2.0,40,36,43,-7,0.900000,38,32,39,-7,0.842105,25.60,17.79,1240.42
3,1995-1996,1995,Segunda División,2,15,GK,Francisco Sánchez Broto,Spain,24,720,7,0,8,0,8,0,0,0.000000,0.000000,DNQ,DNQ,DNQ,NaN,2,6,4,2.0,40,36,43,-7,0.900000,38,32,39,-7,0.842105,25.60,17.79,1240.42
4,1995-1996,1995,Segunda División,2,15,DF,Pascual Donat,Spain,27,2563,30,1,31,2,33,1,1,2563.000000,33.000000,DNQ,DNQ,DNQ,NaN,2,6,4,2.0,40,36,43,-7,0.900000,38,32,39,-7,0.842105,25.60,17.79,1240.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,2021-2022,2021,La Liga,1,7,FW,Dani Raba,Spain,26,328,0,7,3,9,12,1,0,328.000000,12.000000,Champions League,1,Semis,4/5,22,7,16,-9.0,54,98,57,41,1.814815,38,63,37,26,1.657895,25.41,21.79,1379.08
930,2021-2022,2021,La Liga,1,7,FW,Yéremi Pino,Spain,19,2225,21,10,28,12,40,7,6,317.857143,5.714286,Champions League,1,Semis,4/5,22,7,16,-9.0,54,98,57,41,1.814815,38,63,37,26,1.657895,25.41,21.79,1379.08
931,2021-2022,2021,La Liga,1,7,FW,Boulaye Dia,Senegal,25,1653,13,12,17,18,35,7,5,236.142857,5.000000,Champions League,1,Semis,4/5,22,7,16,-9.0,54,98,57,41,1.814815,38,63,37,26,1.657895,25.41,21.79,1379.08
932,2021-2022,2021,La Liga,1,7,FW,Samu Chukwueze,Nigeria,22,1766,12,15,16,22,38,7,3,252.285714,5.428571,Champions League,1,Semis,4/5,22,7,16,-9.0,54,98,57,41,1.814815,38,63,37,26,1.657895,25.41,21.79,1379.08


In [3]:
data.isna().sum()

season                               0
season_start_year                    0
league                               0
division                             0
league_finish                        0
position                             0
name                                 0
nationality                          0
age                                  0
minutes_(total)                      0
starting_appearances_(league)        0
substitute_appearances_(league)      0
starting_appearances_(total)         0
substitute_appearances_(total)       0
total_appearances                    0
goals_(total)                        0
goals_(league)                       0
g/minutes_(total)                    0
g/appearances_(total)                0
european_competition                 0
tier                                 0
performance                          0
round_of_tournament                291
homegrown_players                    0
transfers_in                         0
transfers_out            

In [4]:
data = data.drop(['season', 'name', 'performance', 'nationality'], axis=1) 

In [5]:
data["tier"] = data["tier"].replace({"DNQ":0}).astype(np.int64)

In [6]:
data["round_of_tournament"].unique()

array([nan, '5/5', '5/6', '4/6', '4/5', '3/5', '2/6', '1/5', '3/6', '6/6'],
      dtype=object)

In [7]:
def clean_tournament(rounds):
        
    if pd.isna(rounds):
        return 0 
    if rounds == '1/5':
        return 20
    if rounds == '2/5': 
        return 40
    if rounds == '3/5':
        return 60
    if rounds == '4/5': 
        return 80
    if rounds == '5/5':
        return 100
    if rounds == '1/6':
        return 17
    if rounds == '2/6': 
        return 34
    if rounds == '3/6':
        return 50
    if rounds == '4/6': 
        return 67
    if rounds == '5/6':
        return 84
    if rounds == '6/6':
        return 100

In [8]:
data["round_of_tournament"] = data["round_of_tournament"].apply(clean_tournament)

### X-y Splits and Train_Test Splits

In [10]:
X = data.drop(['homegrown_players'], axis=1)  
y = data['homegrown_players']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
X_train_num = X_train.select_dtypes(np.number)
X_test_num  = X_test.select_dtypes(np.number)

X_train_cat = X_train.select_dtypes(object)
X_test_cat  = X_test.select_dtypes(object)

In [13]:
print(X_train_num.shape)
print(X_test_num.shape)
print(X_train_cat.shape)
print(X_test_cat.shape)
print(y_train.shape)
print(y_test.shape)

(653, 32)
(281, 32)
(653, 3)
(281, 3)
(653,)
(281,)


In [14]:
y_train.isna().sum()

0

# Scaling

In [29]:
transformer = MinMaxScaler()
transformer.fit(X_train_num) 

with open("scalers", "wb") as file: 
    pickle.dump(transformer, file)

X_train_normalized = transformer.transform(X_train_num)
X_test_normalized  = transformer.transform(X_test_num)

X_train_norm = pd.DataFrame(X_train_normalized, columns = X_train_num.columns)
X_test_norm = pd.DataFrame(X_test_normalized, columns = X_test_num.columns)

In [30]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(X_train_cat)

with open("encoders", "wb") as file: 
    pickle.dump(encoder, file)
    
X_train_cat_enc = encoder.transform(X_train_cat).toarray()
X_test_cat_enc  = encoder.transform(X_test_cat).toarray()

X_train_cat_enc = pd.DataFrame(X_train_cat_enc, columns=encoder.get_feature_names_out())
X_test_cat_enc = pd.DataFrame(X_test_cat_enc, columns=encoder.get_feature_names_out())

In [17]:
X_train_final = pd.concat([X_train_norm, X_train_cat_enc], axis = 1)

In [18]:
X_test_final = pd.concat([X_test_norm, X_test_cat_enc], axis = 1)

# Training the Model

In [32]:
model = LinearRegression()
model.fit(X_train_final, y_train)

with open("models", "wb") as file: 
    pickle.dump(model, file)

In [20]:
y_pred_train = model.predict(X_train_final) 
y_pred_test = model.predict(X_test_final) 
r2_score_train = r2_score(y_pred_train, y_train)
r2_score_test = r2_score(y_pred_test, y_test)

In [21]:
r2 = r2_score(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)

print("The r2 of the model is: {:.3f}".format(r2))
print("The mean absolute error of the model is: {:.3f}".format(mae))
print("The mean squared error of the model is: {:.3f}".format(mse))
print("The root mean squared error of the model is: {:.3f}".format(rmse))

The r2 of the model is: 0.995
The mean absolute error of the model is: 0.375
The mean squared error of the model is: 0.242
The root mean squared error of the model is: 0.492


In [22]:
r2 = r2_score(y_train, y_pred_train)
mae = mean_absolute_error(y_train, y_pred_train)
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)


print("The r2 of the model is: {:.3f}".format(r2))
print("The mean absolute error of the model is: {:.3f}".format(mae))
print("The mean squared error of the model is: {:.3f}".format(mse))
print("The root mean squared error of the model is: {:.3f}".format(rmse))

The r2 of the model is: 0.996
The mean absolute error of the model is: 0.351
The mean squared error of the model is: 0.218
The root mean squared error of the model is: 0.467


In [23]:
y_pred = model.predict(X_test_final)

In [24]:
predictions = model.predict(X_test_final)
model.score(X_test_final, y_test)

0.994992063890612

In [45]:
#Try with the same dataset to do a KNN Regressor- find best distance 
knn = KNeighborsRegressor(n_neighbors=6, weights='distance', algorithm='auto', leaf_size=30)
knn.fit(X_train_final, y_train) 

with open("models", "wb") as file: 
    pickle.dump(knn, file)

In [46]:
y_pred_imputed = knn.predict(X_test_final)

# In the below 

In [47]:
rmse_val = [] 
for K in range(30):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K, weights='distance')

    model.fit(X_train_final, y_train)  
    y_pred_imputed = model.predict(X_test_final)
    error = np.sqrt(mean_squared_error(y_test,y_pred_imputed))
    rmse_val.append(error) 
    print('RMSE value for k= ' , K , 'is:', error)

RMSE value for k=  1 is: 0.9280139893411151
RMSE value for k=  2 is: 0.7305435604810322
RMSE value for k=  3 is: 0.7198194710835588
RMSE value for k=  4 is: 0.8248172546827165
RMSE value for k=  5 is: 0.973222648878086
RMSE value for k=  6 is: 1.095677920804926
RMSE value for k=  7 is: 1.2323124613648828
RMSE value for k=  8 is: 1.3149164136819311
RMSE value for k=  9 is: 1.3996985143098322
RMSE value for k=  10 is: 1.4557705280586037
RMSE value for k=  11 is: 1.490298475707009
RMSE value for k=  12 is: 1.546689661030629
RMSE value for k=  13 is: 1.5996172751010989
RMSE value for k=  14 is: 1.6296871999319054
RMSE value for k=  15 is: 1.679065411975571
RMSE value for k=  16 is: 1.7367391412339022
RMSE value for k=  17 is: 1.767865749484137
RMSE value for k=  18 is: 1.7806215491237685
RMSE value for k=  19 is: 1.798432909580443
RMSE value for k=  20 is: 1.8311167031071869
RMSE value for k=  21 is: 1.84399161149538
RMSE value for k=  22 is: 1.8567029627192408
RMSE value for k=  23 is: 1.

In [48]:
r2 = r2_score(y_test, y_pred_imputed)
print("The r2 of the model is: {:.3f}".format(r2))

The r2 of the model is: 0.921
